In [64]:
import numpy as np

## Task1

In [65]:
blosum50 =np.loadtxt('blosum50.txt', dtype=int)
print(blosum50)

[[ 5 -2 -1 -2 -1 -1 -1  0 -2 -1 -2 -1 -1 -3 -1  1  0 -3 -2  0]
 [-2  7 -1 -2 -4  1  0 -3  0 -4 -3  3 -2 -3 -3 -1 -1 -3 -1 -3]
 [-1 -1  7  2 -2  0  0  0  1 -3 -4  0 -2 -4 -2  1  0 -4 -2 -3]
 [-2 -2  2  8 -4  0  2 -1 -1 -4 -4 -1 -4 -5 -1  0 -1 -5 -3 -4]
 [-1 -4 -2 -4 13 -3 -3 -3 -3 -2 -2 -3 -2 -2 -4 -1 -1 -5 -3 -1]
 [-1  1  0  0 -3  7  2 -2  1 -3 -2  2  0 -4 -1  0 -1 -1 -1 -3]
 [-1  0  0  2 -3  2  6 -3  0 -4 -3  1 -2 -3 -1 -1 -1 -3 -2 -3]
 [ 0 -3  0 -1 -3 -2 -3  8 -2 -4 -4 -2 -3 -4 -2  0 -2 -3 -3 -4]
 [-2  0  1 -1 -3  1  0 -2 10 -4 -3  0 -1 -1 -2 -1 -2 -3  2 -4]
 [-1 -4 -3 -4 -2 -3 -4 -4 -4  5  2 -3  2  0 -3 -3 -1 -3 -1  4]
 [-2 -3 -4 -4 -2 -2 -3 -4 -3  2  5 -3  3  1 -4 -3 -1 -2 -1  1]
 [-1  3  0 -1 -3  2  1 -2  0 -3 -3  6 -2 -4 -1  0 -1 -3 -2 -3]
 [-1 -2 -2 -4 -2  0 -2 -3 -1  2  3 -2  7  0 -3 -2 -1 -1  0  1]
 [-3 -3 -4 -5 -2 -4 -3 -4 -1  0  1 -4  0  8 -4 -3 -2  1  4 -1]
 [-1 -3 -2 -1 -4 -1 -1 -2 -2 -3 -4 -1 -3 -4 10 -1 -1 -4 -3 -3]
 [ 1 -1  1  0 -1  0 -1  0 -1 -3 -3  0 -2 -3 -1  5  2 -4

In [66]:
blosum_index_map = dict(zip(['A', 'R', 'N', 'D', 'C', 'Q', 'E', 'G', 'H', 'I', 'L', 'K', 'M', 'F', 'P', 'S', 'T', 'W', 'Y', 'V'],[i for i in range(len(blosum50))]))

def get_blosum_score(a,b):
    
    return blosum50[blosum_index_map[a],blosum_index_map[b]]

print(get_blosum_score('H','P'))


-2


In [67]:
def needleman_wunsch(protein1,protein2):
    d = -8
    width = len(protein1)+1
    heigh = len(protein2)+1
    matrix = np.zeros((heigh,width))
    path = []
    for i in range(1,heigh):
        matrix[i,0] = matrix[i-1,0]+d
    for j in range(1,width):
        matrix[0,j] = matrix[0,j-1]+d
    for i in range(1,heigh):
        for j in range(1,width):
            matrix[i,j] = max(matrix[i-1,j]+d,matrix[i,j-1]+d,matrix[i-1,j-1]+get_blosum_score(protein1[j-1],protein2[i-1]))
    i = heigh-1
    j = width-1
    # path add point coordinate and matrix value and add how to get this point
    while i>0 or j>0:
        if i>0 and j>0 and matrix[i,j] == matrix[i-1,j-1]+get_blosum_score(protein1[j-1],protein2[i-1]):
            path.append((i-1,j-1,matrix[i,j],'diagonal'))
            i-=1
            j-=1
        elif i>0 and matrix[i,j] == matrix[i-1,j]+d:
            path.append((i-1,j,matrix[i,j],'up'))
            i-=1
        elif j>0 and matrix[i,j] == matrix[i,j-1]+d:
            path.append((i,j-1,matrix[i,j],'left'))
            j-=1
            
    # get protein1 and protein2 alignment 
    protein1_alignment = ''
    protein2_alignment = ''
    for i in range(len(path)-1,-1,-1):
        if path[i][3] == 'diagonal':
            protein1_alignment += protein1[path[i][1]]
            protein2_alignment += protein2[path[i][0]]
        elif path[i][3] == 'up':
            protein1_alignment += '-'
            protein2_alignment += protein2[path[i][0]]
        elif path[i][3] == 'left':
            protein1_alignment += protein1[path[i][1]]
            protein2_alignment += '-'
    return path,matrix, protein1_alignment, protein2_alignment
     
print(needleman_wunsch('HEAGAWGHEE','PAWHEAE'))
print(needleman_wunsch('SALPQPTTPVSSFTSGSMLGRTDTALTNTYSAL','PSPTMEAVTSVEASTASHPHSTSSYFATTYYHLY'))

    

([(6, 9, 1.0, 'diagonal'), (5, 9, -5.0, 'up'), (4, 8, 3.0, 'diagonal'), (3, 7, -3.0, 'diagonal'), (3, 6, -13.0, 'left'), (2, 5, -5.0, 'diagonal'), (1, 4, -20.0, 'diagonal'), (1, 3, -25.0, 'left'), (0, 2, -17.0, 'diagonal'), (0, 1, -16.0, 'left'), (0, 0, -8.0, 'left')], array([[  0.,  -8., -16., -24., -32., -40., -48., -56., -64., -72., -80.],
       [ -8.,  -2.,  -9., -17., -25., -33., -41., -49., -57., -65., -73.],
       [-16., -10.,  -3.,  -4., -12., -20., -28., -36., -44., -52., -60.],
       [-24., -18., -11.,  -6.,  -7., -15.,  -5., -13., -21., -29., -37.],
       [-32., -14., -18., -13.,  -8.,  -9., -13.,  -7.,  -3., -11., -19.],
       [-40., -22.,  -8., -16., -16.,  -9., -12., -15.,  -7.,   3.,  -5.],
       [-48., -30., -16.,  -3., -11., -11., -12., -12., -15.,  -5.,   2.],
       [-56., -38., -24., -11.,  -6., -12., -14., -15., -12.,  -9.,   1.]]), 'HEAGAWGHE-E', '--P-AW-HEAE')
([(33, 33, 7.0, 'up'), (32, 32, 15.0, 'diagonal'), (31, 31, 10.0, 'diagonal'), (30, 30, 12.0, 'dia

## Task2

In [68]:
def smith_waterman(protein1,protein2):
    d = -8
    height = len(protein2)+1
    width = len(protein1)+1
    
    matrix = np.zeros((height,width))
    # smith waterman algorithm achieve
    for i in range(1,height):
        for j in range(1,width):
            matrix[i,j] = max(0,matrix[i-1,j]+d,matrix[i,j-1]+d,matrix[i-1,j-1]+get_blosum_score(protein1[j-1],protein2[i-1]))
    # get max value and max value coordinate
    max_value = 0
    max_value_coordinate = (0,0)
    for i in range(height):
        for j in range(width):
            if matrix[i,j]>max_value:
                max_value = matrix[i,j]
                max_value_coordinate = (i,j)
    # get path
    path = []
    i = max_value_coordinate[0]
    j = max_value_coordinate[1]
    while True:
        if i>0 and j>0 and matrix[i,j] == matrix[i-1,j-1]+get_blosum_score(protein1[j-1],protein2[i-1]):
            path.append((i-1,j-1,matrix[i,j],'diagonal'))
            i-=1
            j-=1
        elif i>0 and matrix[i,j] == matrix[i-1,j]+d:
            path.append((i-1,j,matrix[i,j],'up'))
            i-=1
        elif j>0 and matrix[i,j] == matrix[i,j-1]+d:
            path.append((i,j-1,matrix[i,j],'left'))
            j-=1
        else:
            break
    # get protein1 and protein2 alignment
    protein1_alignment = ''
    protein2_alignment = ''
    for i in range(len(path)-1,-1,-1):
        if path[i][3] == 'diagonal':
            protein1_alignment += protein1[path[i][1]]
            protein2_alignment += protein2[path[i][0]]
        elif path[i][3] == 'up':
            protein1_alignment += '-'
            protein2_alignment += protein2[path[i][0]]
        elif path[i][3] == 'left':
            protein1_alignment += protein1[path[i][1]]
            protein2_alignment += '-'
    return path,matrix, protein1_alignment, protein2_alignment

print(smith_waterman('HEAGAWGHEE','PAWHEAE'))
print(smith_waterman( "MQNSHSGVNQLGGVFVNGRPLPDSTRQKIVELAHSGARPCDISRILQVSNGCVSKILGRY","TDDECHSGVNQLGGVFVGGRPLPDSTRQKIVELAHSGARPCDISRI"))


([(4, 8, 28.0, 'diagonal'), (3, 7, 22.0, 'diagonal'), (3, 6, 12.0, 'left'), (2, 5, 20.0, 'diagonal'), (1, 4, 5.0, 'diagonal')], array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  5.,  0.,  5.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  2.,  0., 20., 12.,  4.,  0.,  0.],
       [ 0., 10.,  2.,  0.,  0.,  0., 12., 18., 22., 14.,  6.],
       [ 0.,  2., 16.,  8.,  0.,  0.,  4., 10., 18., 28., 20.],
       [ 0.,  0.,  8., 21., 13.,  5.,  0.,  4., 10., 20., 27.],
       [ 0.,  0.,  6., 13., 18., 12.,  4.,  0.,  4., 16., 26.]]), 'AWGHE', 'AW-HE')
([(45, 44, 273.0, 'diagonal'), (44, 43, 268.0, 'diagonal'), (43, 42, 261.0, 'diagonal'), (42, 41, 256.0, 'diagonal'), (41, 40, 251.0, 'diagonal'), (40, 39, 243.0, 'diagonal'), (39, 38, 230.0, 'diagonal'), (38, 37, 220.0, 'diagonal'), (37, 36, 213.0, 'diagonal'), (36, 35, 208.0, 'diagonal'), (35, 34, 200.0, 'diagonal'), (34, 33, 195.0, 'di

## Task3 BLAST algorithm

CLUSTAL O(1.2.4) multiple sequence alignment


tr|O96791|O96791_DROME      NVIAMRNLPCLGTAGGSGLGGIAGKPSPTMEAVEASTASHPHSTSSYFATTYYHLTDDEC	60
sp|P63015|PAX6_MOUSE        --------------------------------------------------------MQNS	4
                                                                                     ::.

tr|O96791|O96791_DROME      HSGVNQLGGVFVGGRPLPDSTRQKIVELAHSGARPCDISRI-------------------	101
sp|P63015|PAX6_MOUSE        HSGVNQLGGVFVNGRPLPDSTRQKIVELAHSGARPCDISRILQVSNGCVSKILGRYYETG	64
                            ************.****************************                   

tr|O96791|O96791_DROME      ------------------------------------------------------------	101
sp|P63015|PAX6_MOUSE        SIRPRAIGGSKPRVATPEVVSKIAQYKRECPSIFAWEIRDRLLSEGVCTNDNIPSVSSIN	124
                                                                                        

tr|O96791|O96791_DROME      ------------------------------------------------------------	101
sp|P63015|PAX6_MOUSE        RVLRNLASEKQQMGADGMYDKLRMLNGQTGSWGTRPGWYPGTSVPGQPTQDGCQQQEGGG	184
                                                                                        

tr|O96791|O96791_DROME      ------------------------------------------------------------	101
sp|P63015|PAX6_MOUSE        ENTNSISSNGEDSDEAQMRLQLKRKLQRNRTSFTQEQIEALEKEFERTHYPDVFARERLA	244
                                                                                        

tr|O96791|O96791_DROME      ------------------------------------------------------------	101
sp|P63015|PAX6_MOUSE        AKIDLPEARIQVWFSNRRAKWRREEKLRNQRRQASNTPSHIPISSSFSTSVYQPIPQPTT	304
                                                                                        

tr|O96791|O96791_DROME      ------------------------------------------------------------	101
sp|P63015|PAX6_MOUSE        PVSSFTSGSMLGRTDTALTNTYSALPPMPSFTMANNLPMQPPVPSQTSSYSCMLPTSPSV	364
                                                                                        

tr|O96791|O96791_DROME      ----------------------------------------------------------	101
sp|P63015|PAX6_MOUSE        NGRSYDTYTPPHMQTHMNSQPMGTSGTTSTGLISPGVSVPVQVPGSEPDMSQYWPRLQ	422

## Task4

In [69]:

state = {'AT','CG'}
emission_prob = {'AT':{'A':0.2698,'T':0.3237,'C':0.2080,'G':0.1985},'CG':{'A':0.2495,'T':0.2079,'C':0.2478,'G':0.2984}}
tranistion_prob = {'AT':{'AT':0.9998,'CG':0.0002},'CG':{'AT':0.0003,'CG':0.9997}}

# program the following HMM to generate CG rich regions

def 



ValueError: probabilities do not sum to 1

## Task5


In [ ]:
# viterbi algorithm

def viterbi_algorithm():
    

TypeError: unhashable type: 'slice'